# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "/users/amralwakeal/Desktop/Course/untitled_folder/Homework/Homework/Python_API/output_data/cities.csv"
city = pd.read_csv(file)
city.dropna(inplace = True) 
city.info(5)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 0 to 547
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     544 non-null    int64  
 1   City        544 non-null    object 
 2   Cloudiness  544 non-null    int64  
 3   Country     544 non-null    object 
 4   Date        544 non-null    int64  
 5   Humidity    544 non-null    int64  
 6   Lat         544 non-null    float64
 7   Lng         544 non-null    float64
 8   Max Temp    544 non-null    float64
 9   Wind Speed  544 non-null    float64
dtypes: float64(4), int64(4), object(2)
memory usage: 46.8+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# locations
locations = city[["Lat", "Lng"]]

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Gmap configuration

gmaps.configure(api_key=g_key)

# Heat map and and heatmap layer 
fig = gmaps.figure()



# create layer 
heat_layer = gmaps.heatmap_layer(locations, weights=city['Humidity'], dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)

# Add the layer to the map

fig 




Figure(layout=FigureLayout(height='420px'))

In [4]:
ideal_df = pd.DataFrame(city[(city['Max Temp'] > 70) | (city['Max Temp'] < 80)])
ideal_wind = pd.DataFrame(ideal_df[(ideal_df['Wind Speed'] < 10) & (ideal_df['Cloudiness'] == 0)])
ideal_wind.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 3 to 545
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     57 non-null     int64  
 1   City        57 non-null     object 
 2   Cloudiness  57 non-null     int64  
 3   Country     57 non-null     object 
 4   Date        57 non-null     int64  
 5   Humidity    57 non-null     int64  
 6   Lat         57 non-null     float64
 7   Lng         57 non-null     float64
 8   Max Temp    57 non-null     float64
 9   Wind Speed  57 non-null     float64
dtypes: float64(4), int64(4), object(2)
memory usage: 4.9+ KB


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
for index, row in ideal_wind.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    location = f"{lat},{lng}"
    target_coordinates = location
    target_radius = 5000
    target_type = "hotel"
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
    

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    #print the name_address url
    

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_wind.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [6]:
ideal_wind.describe()

,City_ID,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
count,57.000000,57.0,5.700000e+01,57.000000,57.000000,57.000000,57.000000,57.000000
mean,268.859649,0.0,1.558379e+09,58.070175,22.411930,56.826491,59.631053,5.920877
std,165.063888,0.0,7.682837e+01,21.817309,37.701333,74.926006,19.741257,2.415585
min,3.000000,0.0,1.558378e+09,11.000000,-45.580000,-105.250000,4.920000,0.780000
25%,139.000000,0.0,1.558379e+09,44.000000,-20.630000,5.880000,48.200000,4.470000
50%,259.000000,0.0,1.558379e+09,59.000000,35.170000,59.220000,62.600000,6.200000
75%,416.000000,0.0,1.558379e+09,72.000000,52.210000,113.300000,72.240000,8.050000
max,545.000000,0.0,1.558379e+09,93.000000,71.970000,178.420000,94.200000,9.440000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_wind
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 3 to 545
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     57 non-null     int64  
 1   City        57 non-null     object 
 2   Cloudiness  57 non-null     int64  
 3   Country     57 non-null     object 
 4   Date        57 non-null     int64  
 5   Humidity    57 non-null     int64  
 6   Lat         57 non-null     float64
 7   Lng         57 non-null     float64
 8   Max Temp    57 non-null     float64
 9   Wind Speed  57 non-null     float64
 10  Hotel Name  57 non-null     object 
dtypes: float64(4), int64(4), object(3)
memory usage: 7.8+ KB


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [9]:
hotel_locations = hotel_df[["Lat", "Lng"]]

heatmap_layer = gmaps.heatmap_layer(
    hotel_locations, max_intensity=30, point_radius=3.0)

#heat_layer = gmaps.symbol_layer(hotel_locations, fill_color='green', stroke_color='green', scale=2)

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

fig = gmaps.figure()
#fig.add_layer(marker_layer)

fig.add_layer(heatmap_layer)

fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))